## Assignment 3 Question 2

Is it possible to predict ELO of a player based on context of a potential en passant move?

Only investigating the player who is in the position of making the en passant move.

Lila See FDS PCA solutions for 3 good points for what PCA is good for - include in report?

#### Import Libraries

In [43]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import chess
import chess.pgn
import io
import re
import time
# import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#### Import CSV file

66879 entries in dataframe

In [44]:
# Load CSV into a big dataframe
CHESS_DATA_LOCATION = "data/club_games_data.csv"
chess_data = pd.read_csv(CHESS_DATA_LOCATION)

#### Dataframe info

Using:
print(chess_data.dtypes)
print(chess["rules"].unique())


Chess rules:
['chess' 'chess960' 'threecheck' 'crazyhouse' 'kingofthehill']

Time control:
['1/259200' '1/172800' '1800' '1/86400' '1/432000' '1/604800' '600'
 '120+1' '900+10' '300' '180+2' '3600+5' '2700+45' '3600' '1/1209600'
 '180' '600+10' '60' '480+3' '300+5' '420+3' '600+5' '600+2' '1200' '30'
 '60+1' '120' '1500+3' '900+2' '1500+5' '1500+10' '1/864000' '900' '300+2'
 '1500' '7200' '300+1' '5400' '3600+60' '2700+30' '3480+45' '10' '2700+10'
 '15' '2700' '3600+20' '4500' '4200' '900+5' '1800+10' '2700+5' '480+5'
 '1800+30' '300+3' '600+1' '1800+5' '420+5' '5400+30' '240+10' '420' '303'
 '60+10']

 Time class:
['daily' 'rapid' 'bullet' 'blitz']

#### Clean data

Undeveloped board shouldn't matter if we're filtering games for potential ep

Same for draws

Can filter our time class if are looking at time controls

After making a new move_list column, should we drop the pgn column?


Variables we are considering when predicting ELO (for a player who could potentially make en passant move) are:
- Colour (boolean)
- Did they take the en passant? (boolean)
- Does their choice on taking/not taking support them if gaining an advantage? (numerical value for how much of an advantage it gives)
- Time taken to decide to capture/not capture en passant (... whatever can be time, a number in seconds ig)
- Is the game rated? (boolean)


To do:
- Work out which colour is making the potential en passant move, add a column to dataframe detailing this
- Make dataframe columns for other variables
- Apply PCA reduction

In [45]:
# Drops rows if any value is a NaN (data is clean so it doesn't do anything)
chess_data.dropna(axis = 0, how = 'any')

# Filter out alternative rules like chess960 etc
# Important this comes before the drop irrelevant columns line
chess_data = chess_data[chess_data['rules'] == "chess"]

# Save PGN column from dataframe
full_pgn = chess_data['pgn']

def get_moves(entry):
    '''
    Retrive series of moves in a game when given the whole full_pgn entry
    '''
    pgn = entry.splitlines()[-1]
    return pgn

# Add list of moves (string) as a new column to dataframe
chess_data['move_list'] = full_pgn.apply(get_moves)

# Drop irrelevant columns
chess_data = chess_data.drop(['time_class', 'white_username', 'black_username','white_id', 'black_id', 'white_result', 'black_result', 'rules'], axis=1)

print(chess_data.head())


   white_rating  black_rating time_control  rated  \
0          1708          1608     1/259200   True   
1          1726          1577     1/172800   True   
2          1727           842     1/172800   True   
3           819          1727     1/172800   True   
4          1729          1116     1/172800   True   

                                                 fen  \
0  r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R ...   
1       8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33   
2  rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/...   
3  r3kb1r/pp3ppp/3p1n2/2pKp3/P3P3/1P6/4qP1P/QNB5 ...   
4  r3b2r/pp6/2pPpR1k/4n3/2P3Q1/3B4/PP4PP/R5K1 b -...   

                                                 pgn  \
0  [Event "Enjoyable games 2 - Round 1"]\n[Site "...   
1  [Event "Rapid Rats - Board 5"]\n[Site "Chess.c...   
2  [Event "CHESS BOARD CLASH - Round 1"]\n[Site "...   
3  [Event "CHESS BOARD CLASH - Round 1"]\n[Site "...   
4  [Event "CHESS BOARD CLASH - Round 1"]\n[Site "...   

       

#### Import chess info
https://python-chess.readthedocs.io/en/latest/core.html#chess.Board.san


#### En Passant functions
- has_legal_en_passant() tests if en passant capturing would actually be possible on the next move.
- has_pseudo_legal_en_passant()
- has_legal_en_passant()
- is_en_passant(move: Move) Checks if the given pseudo-legal move is an en passant capture.




Use StringIO to parse games from a string.

```python
import io
pgn = io.StringIO("1. e4 e5 2. Nf3 *")
game = chess.pgn.read_game(pgn)
```

In [46]:
# # chess testing
# fen = "6k1/6p1/p3r2p/3Q4/4NK2/P1r2q2/8/8 w - -"
# board = chess.Board(fen)
# # print(board)

# pgntext = "1. d4 d5 2. e3 c6 3. c4 dxc4 4. Bxc4 b5 5. Bb3 a5 6. Qf3 Bb7 7. Bxf7+ Kd7 8. Qf5+ Kc7 9. Nf3 Nh6 10. Qe5+ Kb6 11. a4 bxa4 12. Nc3 Ng4 13. Nxa4+ Ka6 14. Bc4+ Ka7 15. Qc5+ 1-0"

# pgn_en = "1. e4 h5 2. e5 f5 3. exf6 b5"

# pgn_en2 = "1. d4 h5 2. d5 e5 3. h3"

# # pgn = io.StringIO(chess_data['pgn'][134])
# pgn = io.StringIO(pgn_en)
# game = chess.pgn.read_game(pgn)
# board = game.board()

# # print(game)

# check = False

def check_pgn_opportunity(pgn_in):
    '''
    Checks PGN for whether opportunity for EP happened in the game.
    '''
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    precheck = False                # Is en passant possible
    moved = False                   # Was en passant moved?

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        turn = board.turn
        san = board.san(move)
        move_piece = san[0]

        match move_piece:
            # Ignore if not pawn
            case "K" | "Q" | "B" | "N" | "R" | "O":
                board.push(move)
                continue

            # Pawn
            case _:
                # Push the move before checking the board
                board.push(move)
                precheck = board.has_legal_en_passant()

                # Return True the moment a potential ep move has been found
                if precheck:
                    return True
                
    # If ran through every move without ep opportunity then return False
    return False

def check_pgn_happened(pgn_in):
    '''
    Checks PGN for whether EP actually happened in the game.

    Assumes game passed in already has the opportunity for an EP move.
    Returns True if EP was actually the next move.
    '''
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    moved = False                   # Was en passant moved?

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        turn = board.turn
        san = board.san(move)
        move_piece = san[0]

        match move_piece:
            # Ignore if not pawn
            case "K" | "Q" | "B" | "N" | "R" | "O":
                board.push(move)
                continue

            # Pawn
            case _:
                # Check each move to see if it was an ep move
                moved = board.is_en_passant(move)
                board.push(move)

                # If ep actually happened, immediately return True
                if moved:
                    return True
                
    # If ran through every move without ep then return False
    return False

def check_pgn_turn(pgn_in):
    '''
    Checks which player has the opportunity for EP.

    Assumes check_pgn_opportunity is true for this pgn - otherwise None is returned.
    Returns 'White' or 'Black' depending on which player has opportunity for ep
    '''
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    precheck = False                # Is en passant possible
    moved = False                   # Was en passant moved?

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        turn = board.turn
        san = board.san(move)
        move_piece = san[0]

        match move_piece:
            # Ignore if not pawn
            case "K" | "Q" | "B" | "N" | "R" | "O":
                board.push(move)
                continue

            # Pawn
            case _:
                # Find the ep opportunity move
                # Push the move before checking the board
                board.push(move)
                precheck = board.has_legal_en_passant()

                if precheck:
                    if turn:
                        return 'Black'
                    else:
                        return 'White'
                
    # If pgn without ep opportunity was passed in, return None
    return None


# # Add Boolean for whether oppotunity for ep arose as a new column to dataframe
# # ep_turn is True if ep player is white
# for i in range (len(chess_data)):
#     chess_data['ep_opportunity'][i] = check_pgn(chess_data['move_list'][i], False)[0]
#     chess_data['ep_turn'][i] = check_pgn(chess_data['move_list'][i], False)[1]
#     if chess_data['ep_opportunity'][i]:
#         chess_data['ep_happened'][i] = check_pgn(chess_data['move_list'][i], True)[0]



# Save PGN column from dataframe
full_pgn = chess_data['pgn']

# Add Boolean value for whether an ep opportunity arose to dataframe
chess_data['ep_opportunity'] = full_pgn.apply(check_pgn_opportunity)

# Drop games where ep didn't happen



chess_data['ep_happened']





print(chess_data.head())

# # create new series
# ep = [False] * len(chess_data)
# ep_series = pd.Series(ep)

# turn = [""] * len(chess_data)
# ep_turn = pd.Series(turn)

# #print(check_pgn(pgn_en2, True))
# # true is white

# # For each game in chess_data
# for i in range(len(chess_data)):
#     # check if there is e.p. opportunity and store in series if yes
#     checkpgn = check_pgn(chess_data['pgn'][i], False)
#     if (checkpgn[0] == True):
#         ep_series[i] = True
#         if (checkpgn[1] == True):
#             ep_turn[i] = "White"
#         else:
#             ep_turn[i] = "Black"

# # append series to main dataframe
# chess_data['en_passant'] = ep_series
# chess_data['ep_turn'] = ep_turn

   white_rating  black_rating time_control  rated  \
0          1708          1608     1/259200   True   
1          1726          1577     1/172800   True   
2          1727           842     1/172800   True   
3           819          1727     1/172800   True   
4          1729          1116     1/172800   True   

                                                 fen  \
0  r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R ...   
1       8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33   
2  rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/...   
3  r3kb1r/pp3ppp/3p1n2/2pKp3/P3P3/1P6/4qP1P/QNB5 ...   
4  r3b2r/pp6/2pPpR1k/4n3/2P3Q1/3B4/PP4PP/R5K1 b -...   

                                                 pgn  \
0  [Event "Enjoyable games 2 - Round 1"]\n[Site "...   
1  [Event "Rapid Rats - Board 5"]\n[Site "Chess.c...   
2  [Event "CHESS BOARD CLASH - Round 1"]\n[Site "...   
3  [Event "CHESS BOARD CLASH - Round 1"]\n[Site "...   
4  [Event "CHESS BOARD CLASH - Round 1"]\n[Site "...   

       

In [47]:
# new series
ep_acted = pd.Series(ep)

# for each in chess data
for i in range(len(chess_data)):
    # if there is e.p. opportunity, check if the player acted on it store if yes
    if (chess_data['en_passant'][i] == True and check_pgn(chess_data['pgn'][i], True)):
        ep_acted[i] = True

# append series to main dataframe
chess_data['ep_acted'] = ep_acted

KeyError: 'en_passant'

In [ ]:
print("number of E.P. opportunities:")
print(len(chess_data[chess_data['en_passant'] == True]))
print("number of E.P. moves")
print(len(chess_data[chess_data['ep_acted'] == True]))

### Number of games with at least 1 possible en-passant move:
## 4750
(maybe 4913 or even 4945)

### Number of games with an actual en-passant move:
## 1566